In [1]:
# What is this hyper-parameter optimization>
# If we train a linear regression with SGD,
# the parameters of are the slope and the bias
# and hyperparameter is learning rate.
# Lets' say we have a 3 dial combination lock.
# a combination lock has just one key
# but the model has many right answers

### How would you find the best parameters?
### A method would be to evaluate all the combinations
### and see which one improves the metric

In [3]:
# define the besr accuracy to be 0
# if you choose loss as a metric,
# you can  make best loss to be inf (np.inf)
best_accuracy = 0
best_parameters = {"a": 0, "b": 0, "c": 0}

MODEL = None
training_data = None
validation_data = None
# loop over all values for a,b & c
for a in range(1, 11):
    for b in range(1, 11):
        for c in range(1, 11):
            # initialize model with current parameters
            model = MODEL(a, b, c)
            # fit the model
            model.fit(training_data)
            # make predicitions
            preds = model.predict(validation_data)
            # calculate accuracy
            accuracy = metrics.accuracy_score(targets, preds)
            # save params if current accuracy
            # is greater than best accuracy
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_parameters["a"] = a
                best_parameters["b"] = b
                best_parameters["c"] = c


TypeError: 'NoneType' object is not callable

In [ ]:
# Most models parameters are real-valued, and the 
# combinations of different parameters can be infinite.
# ex: RandomForestClassifier have 19 parameters
# all the combinations is going to be infinite


In [4]:
# Grid of Parameters
# A search over the above grid to find the best combination
# is know as GRID SEARCH
# { n_estimators: [100, 200, 250, 400, 500],
#   max_depth: [1, 2, 7, 11, 15],
#   criterion: ["gini", "entropy"] 
# } 
# It must also be noted that if you have k-fold
# cross-validation, you need more loops which 
# implies more time to find the perfect parameters
# Grid Search is there are not very popular
# Example: prediciting mobile phone price range
# rf_grid_search.py


In [7]:
import pandas as pd
df = pd.read_csv("../input/mobile_train.csv")
df.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [9]:
# Use of PIPELINE
# Lets assume that the pipeline u choose is to 
# first apply tf-idf in a semi-supervised manner and then
# use SVD with SVM classifier.
# We have to select components of SVD
# and also need to tune the parameters of SVM
# pipleline_search.py

In [10]:
import numpy as np
import pandas as pd

from functools import partial

from sklearn import ensemble
from sklearn import metrics
from sklearn import model_selection

from skopt import gp_minimize
from skopt import space
from skopt.plots import plot_convergence

def optimize(params, param_names, x, y):
    """
    The main optimization function.
    This function takes all the arguments from the search space
    and training features and targets. It then initializes 
    the models by setting the chosen parameters and runs
    cross-validation and returns a negative accuracy score
    :param params: list of parameters from gp_minimize
    :param param_names: list of param names. order is important
    :param x: training data
    :param y: labels/targets
    :return: negative accuracy after 5 folds
    """
    # convert params to dictionary
    params = dict(zip(param_names, params))

    # initialize model with current parameters
    model = ensemble.RandomForestClassifier(**params)

    # initialize stratified k-fold
    kf = model_selection.StratifiedKFold(n_splits=5)

    # initialize accuracy list
    accuracies = []

    # loop over all folds
    for idx in kf.split(X=x, y=y):
        train_idx, test_idx = idx[0], idx[1]
        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        # fit model for current fold
        model.fit(xtrain, ytrain)

        # create predictions
        preds = model.predict(xtest)

        # calculate and append accuracy
        fold_accuracy = metrics.accuracy_score(
            ytest,
            preds
        )
        accuracies.append(fold_accuracy)
    
    # return negative accuracys
    return -1 * np.mean(accuracies)

if __name__ == "__main__":
    # read the training data
    df = pd.read_csv("/Users/insomni_.ak/Documents/Machine Learning/AAAMLP/AAAMLP/input/mobile_train.csv")

    # features are all columns without price_range
    # note that there is no id column in this dataset
    # here we have training features
    X = df.drop("price_range", axis=1).values
    # and the targets
    y = df.price_range.values

    # define a parameter space
    param_space = [
        # max depth is an integer between 3 and 10
        space.Integer(3, 15, name="max_depth"),
        # n_estimators is an integer between 50 and 1500
        space.Integer(100, 1500, name="n_estimators"),
        # criterion is a category. here we define list of categories
        space.Categorical(["gini", "entropy"], name="criterion"),
        # you can also have Real numbered space and dfine a
        # distribution you want to pick it from
        space.Real(0.01, 1, prior="uniform", name="max_features")
    ]

    # make a list of param  names
    # this has to be same order as the search space
    # inside the main function
    param_names = [
        "max_depth",
        "n_estimators",
        "criterion",
        "max_features"
    ]

    # by using functools partial, i am creating a 
    # new function which has same parameters as the
    # optimize function except for the fact that 
    # only one param, i.e. the "params" parameter is
    # required. this is how gp_minimze expects the 
    # optimization function to be. you can rid of this
    # by reading data inside the optimize function or by
    # defincing the optimize function here.
    optimization_function = partial(
        optimize,
        param_names=param_names,
        x=X,
        y=y
    )

    # now we call gp_minimize from scikit-optimize
    # gp_minimize uses bayesian optimaization for
    # minimization of the optimization function.
    # we need a space of parameters, the function itself,
    # the number of calls/iterations we want to have 
    result = gp_minimize(
        optimization_function,
        dimensions=param_space,
        n_calls=15,
        n_random_starts=10,
        verbose=10
    )

    # create best params dict and print it
    best_params = dict(
        zip(
            param_names,
            result.x
        )
    )
    print(best_params)
    plot_convergence(result)





ModuleNotFoundError: No module named 'skopt'

In [11]:
!pip3 install scikit-optimize

Defaulting to user installation because normal site-packages is not writeable
